In [3]:
# -*- coding: utf-8 -*-
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.operators import PolynomialTensor
from qiskit_nature.second_q.properties import ElectronicDipoleMoment
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC
from qiskit_algorithms.optimizers import COBYLA , SLSQP, L_BFGS_B
from qiskit_algorithms import VQE
from qiskit.primitives import Estimator
from qiskit_algorithms.eigensolvers import NumPyEigensolver
from qiskit.quantum_info import SparsePauliOp, Statevector # Thêm Statevector nếu cần psi0
import numpy as np
from scipy.linalg import expm # Cần cho Magnus
import time # Để đo thời gian
import matplotlib.pyplot as plt # Thêm để có thể vẽ đồ thị nếu cần

# --- Phần code Qiskit của bạn để lấy toán tử ---
print("Chạy phần Qiskit để lấy toán tử...")
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
problem = driver.run()
hamiltonian = problem.hamiltonian

# Thêm moment lưỡng cực vào Hamiltonian
dipole: ElectronicDipoleMoment = problem.properties.electronic_dipole_moment
nuclear_dip = None # Khởi tạo
if dipole is not None:
    nuclear_dip = dipole.nuclear_dipole_moment
    # Cập nhật moment lưỡng cực cho các thành phần x, y, z
    dipole.x_dipole.alpha += PolynomialTensor({"": nuclear_dip[0]})
    dipole.y_dipole.alpha += PolynomialTensor({"": nuclear_dip[1]})
    dipole.z_dipole.alpha += PolynomialTensor({"": nuclear_dip[2]})
    print("Đã thêm moment lưỡng cực hạt nhân.")
else:
    print("Moment lưỡng cực không tồn tại trong problem.")

# Mapping sang qubit
second_q_op = hamiltonian.second_q_op()
mapper = JordanWignerMapper()
qubit_p_op = mapper.map(second_q_op) # H0 dạng SparsePauliOp

print("Xong phần lấy H0 (qubit_p_op).")

# Lấy toán tử moment lưỡng cực Z và mapping
dipole_ops = dipole.second_q_ops()
# Đảm bảo chỉ lấy thành phần Z nếu X, Y không đáng kể hoặc bằng 0
dipole_op_z_fermionic = dipole_ops.get("ZDipole", None) # Lấy toán tử Z
if dipole_op_z_fermionic is None:
     raise ValueError("Không tìm thấy toán tử ZDipole.")
dipole_qubit = mapper.map(dipole_op_z_fermionic) # Dạng SparsePauliOp
print("Xong phần lấy Dipole Z (dipole_qubit).")


# --- Chuyển toán tử sang dạng ma trận ---
print("Chuyển đổi toán tử sang ma trận...")
H_static_mat = qubit_p_op.to_matrix()
D_mat = dipole_qubit.to_matrix()
num_qubits = qubit_p_op.num_qubits
DIM = 2**num_qubits
print(f"Số qubit: {num_qubits}, Kích thước ma trận: {DIM}x{DIM}")

# --- Thiết lập các tham số mô phỏng ---
Gamma = 0.25
E0 = 0.01
T_TOTAL = 100 # *** Giảm T_TOTAL để chạy ví dụ nhanh hơn, 100 sẽ rất lâu ***
DT = 0.1    # *** Giảm DT để tăng độ chính xác, nhưng sẽ lâu hơn ***
NUM_STEPS = int(T_TOTAL / DT)
TIME_POINTS = np.linspace(0, T_TOTAL, NUM_STEPS + 1)
print(f"Tham số mô phỏng: T_total={T_TOTAL}, dt={DT}, Num_steps={NUM_STEPS}")
if NUM_STEPS > 500: # Cảnh báo nếu số bước lớn
    print("CẢNH BÁO: Số bước thời gian lớn, tính toán Magnus bậc 2 có thể rất chậm!")


# --- Định nghĩa hàm trường và Hamiltonian phụ thuộc thời gian ---
def f_lorentzian(t, E0, Gamma):
    """ Hàm xung Lorentzian (theo định nghĩa H_t của bạn) """
    # Tránh chia cho 0 nếu Gamma=0, mặc dù không nên
    if Gamma == 0: return 0.0
    # Lưu ý: Bài báo Wan dùng Lorentz: Gamma / (Gamma^2 + t^2). Bạn dùng /pi?
    # Sử dụng công thức bạn cung cấp trong hàm H_t gốc:
    return (E0 / np.pi) * Gamma / (Gamma**2 + t**2)
    # Nếu muốn dùng xung Gaussian như ví dụ trước:
    # T0 = T_TOTAL / 2
    # return E0 * np.exp(-((t - T0)**2) / (2 * Gamma**2))


def H_matrix(t):
    """ Trả về ma trận Hamiltonian tại thời điểm t """
    # Sử dụng hàm xung Lorentzian bạn định nghĩa
    f_t = f_lorentzian(t, E0, Gamma)
    return H_static_mat + f_t * D_mat

# --- Tính toán trước Giao hoán tử [H0, D] ---
print("Tính toán trước [H0, D]...")
Comm_H0_D = H_static_mat @ D_mat - D_mat @ H_static_mat

# --- Tính toán Khai triển Magnus Bậc 2 ---
print(f"\n--- Bắt đầu tính toán Khai triển Magnus Bậc 2 ---")
print(f"(Tính toán này có thể mất nhiều thời gian với {NUM_STEPS} bước...)")
start_time_magnus = time.time()

Omega1 = np.zeros((DIM, DIM), dtype=complex)
Omega2 = np.zeros((DIM, DIM), dtype=complex)

# --- Tính Omega1 = -i * integral(H(t1) dt1) ---
for i in range(NUM_STEPS):
    t1_mid = TIME_POINTS[i] + DT / 2
    H_t1_mid = H_matrix(t1_mid)
    Omega1 += H_t1_mid * DT
Omega1 *= -1j
print(f"Đã tính Omega1.")

# --- Tính Omega2 = -0.5 * integral_0^T dt1 integral_0^t1 dt2 (f(t2) - f(t1)) * Comm_H0_D ---
integral2_payload = np.zeros((DIM, DIM), dtype=complex)
for i in range(NUM_STEPS): # Vòng lặp ngoài cho t1 (0 đến T-dt)
    t1 = TIME_POINTS[i]
    t1_mid = t1 + DT / 2
    f_t1 = f_lorentzian(t1_mid, E0, Gamma)

    # Tích phân nội tại theo t2 từ 0 đến t1 = TIME_POINTS[i]
    integral_inner = np.zeros((DIM, DIM), dtype=complex)
    for j in range(i + 1): # Vòng lặp trong cho t2 (0 đến i)
        t2 = TIME_POINTS[j]
        t2_mid = t2 + DT / 2
        if t2_mid > t1_mid + 1e-9: continue # Đảm bảo t2 <= t1

        f_t2 = f_lorentzian(t2_mid, E0, Gamma)
        integrand = (f_t2 - f_t1) * Comm_H0_D
        # Chỉ cộng nếu khoảng thời gian [tj, tj+DT] nằm hoàn toàn trong [0, t1]
        # Sử dụng quy tắc trung điểm nên chỉ cần kiểm tra điểm giữa
        integral_inner += integrand * DT

    # Cộng vào tích phân kép
    integral2_payload += integral_inner * DT

    # Chỉ báo tiến trình
    if (i + 1) % max(1, NUM_STEPS // 20) == 0:
        elapsed_time = time.time() - start_time_magnus
        print(f"  Omega2 progress: Step {i+1}/{NUM_STEPS} ({elapsed_time:.1f} s)")


Omega2 = -0.5 * integral2_payload
print(f"Đã tính Omega2.")

# --- Kết hợp các số hạng ---
Omega_approx = Omega1 + Omega2

# --- Kiểm tra tính Anti-Hermitian ---
Omega_dag = Omega_approx.conj().T
anti_herm_diff = np.linalg.norm(Omega_dag + Omega_approx)
print(f"\nKiểm tra Anti-Hermitian cho Omega_Magnus2:")
print(f"||Omega^dagger + Omega|| = {anti_herm_diff:.3e}")
if not np.isclose(anti_herm_diff, 0.0, atol=1e-7): # Dung sai chặt hơn một chút
      print("  CẢNH BÁO: Omega_Magnus2 không hoàn toàn anti-Hermitian.")
      print("           Nguyên nhân có thể do sai số tích phân số (thử DT nhỏ hơn).")
else:
      print("  Omega_Magnus2 gần như anti-Hermitian.")

# --- Tính U(T_TOTAL) cuối cùng bằng hàm mũ ma trận ---
print(f"\nTính U(T_TOTAL) = expm(Omega_approx)...")
U_magnus2 = expm(Omega_approx)
end_time_magnus = time.time()
total_magnus_time = end_time_magnus - start_time_magnus
print(f"Hoàn thành tính toán Magnus Bậc 2 trong {total_magnus_time:.2f} giây.")

# --- Hàm kiểm tra Unitarity ---
def check_unitarity(U, name, tol=1e-8):
    Id = np.eye(U.shape[0], dtype=complex)
    UdU = U.conj().T @ U
    norm_diff = np.linalg.norm(UdU - Id)
    print(f"Kiểm tra Unitarity cho {name}:")
    print(f"  ||U^dagger * U - I|| = {norm_diff:.3e}")
    if np.isclose(norm_diff, 0.0, atol=tol):
        print(f"  {name} là unitary trong giới hạn dung sai {tol}.")
        return True
    else:
        print(f"  CẢNH BÁO: {name} KHÔNG unitary trong giới hạn dung sai {tol}.")
        return False

# --- Kiểm tra Unitarity của kết quả ---
check_unitarity(U_magnus2, "U_Magnus2", tol=1e-7)

print("\nTính toán hoàn tất. Biến U_magnus2 chứa ma trận xấp xỉ cho U(T_TOTAL).")

# --- (Tùy chọn) Tính toán động lực học trạng thái ---
# Nếu bạn muốn xem trạng thái tiến hóa như thế nào
# 1. Lấy trạng thái ban đầu psi0 từ VQE
# estimator_vqe = Estimator() # Cần estimator để chạy VQE nếu chưa có 'res'
# vqe = VQE(estimator_vqe, ansatz, optimizer) # Dùng optimizer tốt nhất từ lần chạy trước
# res = vqe.compute_minimum_eigenvalue(qubit_p_op)
# optimal_params = res.optimal_parameters
# psi0_circuit = ansatz.bind_parameters(optimal_params)
# psi0_vector = Statevector(psi0_circuit).data # Tính statevector ban đầu

# 2. Tính trạng thái cuối cùng
# if 'psi0_vector' in locals():
#     psi_final_magnus = U_magnus2 @ psi0_vector
#     print("\nTrạng thái cuối cùng (xấp xỉ Magnus Bậc 2):")
#     # print(psi_final_magnus) # In ra vector trạng thái
#     print(f"Norm trạng thái cuối: {np.linalg.norm(psi_final_magnus)}") # Nên bằng 1
# else:
#     print("\nKhông thể tính trạng thái cuối do thiếu psi0_vector từ VQE.")

Chạy phần Qiskit để lấy toán tử...
Đã thêm moment lưỡng cực hạt nhân.
Xong phần lấy H0 (qubit_p_op).
Xong phần lấy Dipole Z (dipole_qubit).
Chuyển đổi toán tử sang ma trận...
Số qubit: 4, Kích thước ma trận: 16x16
Tham số mô phỏng: T_total=100, dt=0.1, Num_steps=1000
CẢNH BÁO: Số bước thời gian lớn, tính toán Magnus bậc 2 có thể rất chậm!
Tính toán trước [H0, D]...

--- Bắt đầu tính toán Khai triển Magnus Bậc 2 ---
(Tính toán này có thể mất nhiều thời gian với 1000 bước...)
Đã tính Omega1.
  Omega2 progress: Step 50/1000 (0.0 s)
  Omega2 progress: Step 100/1000 (0.0 s)
  Omega2 progress: Step 150/1000 (0.0 s)
  Omega2 progress: Step 200/1000 (0.1 s)
  Omega2 progress: Step 250/1000 (0.1 s)
  Omega2 progress: Step 300/1000 (0.1 s)
  Omega2 progress: Step 350/1000 (0.2 s)
  Omega2 progress: Step 400/1000 (0.2 s)
  Omega2 progress: Step 450/1000 (0.3 s)
  Omega2 progress: Step 500/1000 (0.3 s)
  Omega2 progress: Step 550/1000 (0.4 s)
  Omega2 progress: Step 600/1000 (0.5 s)
  Omega2 progr

In [2]:
import numpy as np
from scipy.integrate import simps
from scipy.linalg import expm
import matplotlib.pyplot as plt

# Các tham số
Gamma = 0.25
E0 = 0.01
T_total = 100  # fs
dt = 0.1  # fs
n_steps = int(T_total / dt) + 1
times = np.linspace(0, T_total, n_steps)
m_points = 1000  # Số điểm tích phân Simpson

# Hàm trường ngoài V(t)
def V_t(t, Gamma, E0, dipole_matrix):
    coefficient = (E0 / np.pi) * (Gamma / (Gamma**2 + t**2))
    return coefficient * dipole_matrix

# Hàm H(t)
def H_t(t, H_static, dipole_matrix, Gamma, E0):
    return H_static + V_t(t, Gamma, E0, dipole_matrix)

# Hàm tính commutator
def commutator(A, B):
    return A @ B - B @ A

# Hàm kiểm tra sai số đơn vị tính
def check_unitary_error(U):
    identity = np.eye(U.shape[0])
    error = np.linalg.norm(U.conj().T @ U - identity, ord='fro')
    return error

# Phương pháp Magnus bậc 2
def compute_U_tj_magnus2(H_static, dipole_matrix, t_j, m_points, Gamma, E0):
    if t_j == 0:
        return np.eye(H_static.shape[0], dtype=complex)
    t_points = np.linspace(0, t_j, m_points)
    
    # Tính số hạng bậc 1: -i ∫ H(s) ds
    H_integral = np.zeros_like(H_static, dtype=complex)
    for i in range(H_static.shape[0]):
        for j in range(H_static.shape[1]):
            H_t_values = [H_t(t, H_static, dipole_matrix, Gamma, E0)[i,j] for t in t_points]
            H_integral[i,j] = simps(H_t_values, t_points)
    Omega1 = -1j * H_integral
    
    # Tính số hạng bậc 2: (-i)^2/2 ∫ ds1 ∫ ds2 [H(s1), H(s2)]
    Omega2 = np.zeros_like(H_static, dtype=complex)
    dt = t_j / (m_points - 1) if m_points > 1 else t_j
    for s1_idx in range(m_points):
        s1 = t_points[s1_idx]
        H_s1 = H_t(s1, H_static, dipole_matrix, Gamma, E0)
        inner_integral = np.zeros_like(H_static, dtype=complex)
        s2_points = t_points[:s1_idx+1]
        for i in range(H_static.shape[0]):
            for j in range(H_static.shape[1]):
                H_s2_values = [H_t(s2, H_static, dipole_matrix, Gamma, E0)[i,j] for s2 in s2_points]
                inner_integral[i,j] = simps(H_s2_values, s2_points)
        comm = commutator(H_s1, inner_integral)
        Omega2 += comm * dt
    Omega2 *= (-1j)**2 / 2
    
    # Tổng Omega
    Omega = Omega1 + Omega2
    U = expm(Omega)
    return U

# Tính U(t_j) và kiểm tra độ chính xác
target_unitaries = []
unitary_errors = []

for t_j in times:
    U_tj = compute_U_tj_magnus2(H_static, dipole_matrix, t_j, m_points, Gamma, E0)
    error = check_unitary_error(U_tj)
    target_unitaries.append(U_tj)
    unitary_errors.append(error)

# In sai số
avg_error = np.mean(unitary_errors)
max_error = np.max(unitary_errors)
print("Magnus bậc 2:")
print(f"  Sai số đơn vị tính trung bình: {avg_error:.2e}")
print(f"  Sai số đơn vị tính tối đa: {max_error:.2e}")

# Vẽ sai số
plt.figure(figsize=(6, 4))
plt.plot(times, unitary_errors, label='Magnus 2')
plt.xlabel("Time (fs)")
plt.ylabel("Unitary Error ($\|U^\dagger U - I\|_F$)")
plt.yscale('log')
plt.title("Unitary Error")
plt.legend()
plt.grid(True)
plt.show()

# Lưu target_unitaries cho GA-VQA
chosen_unitaries = target_unitaries

NameError: name 'H_static' is not defined

In [4]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
problem = driver.run()
hamiltonian = problem.hamiltonian

# Thêm moment lưỡng cực vào Hamiltonian
import numpy as np 
from qiskit_nature.second_q.operators import PolynomialTensor
from qiskit_nature.second_q.properties import ElectronicDipoleMoment


dipole: ElectronicDipoleMoment = problem.properties.electronic_dipole_moment

if dipole is not None:
    nuclear_dip = dipole.nuclear_dipole_moment
    # Cập nhật moment lưỡng cực cho các thành phần x, y, z
    dipole.x_dipole.alpha += PolynomialTensor({"": nuclear_dip[0]})
    dipole.y_dipole.alpha += PolynomialTensor({"": nuclear_dip[1]})
    dipole.z_dipole.alpha += PolynomialTensor({"": nuclear_dip[2]})
    print("Đã thêm moment lưỡng cực vào Hamiltonian.")
else:
    print("Moment lưỡng cực không tồn tại trong problem.")

# Kiểm tra coefficients của Hamiltonian
coefficients = hamiltonian.electronic_integrals
print("Coefficients của Hamiltonian:", coefficients.alpha)

second_q_op = hamiltonian.second_q_op()
print(second_q_op)

from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()
qubit_p_op = mapper.map(second_q_op)
print(qubit_p_op)

from qiskit_nature.second_q.circuit.library import HartreeFock, UCC

ansatz = UCC(
    num_spatial_orbitals=2,
    num_particles=[1,1],
    excitations='sd',
    qubit_mapper=mapper,
    initial_state=HartreeFock(
        num_spatial_orbitals=2,
        num_particles=[1,1],
        qubit_mapper=mapper,
    ),
    reps=1,

)

from qiskit import *
from qiskit_algorithms.optimizers import COBYLA , SLSQP, L_BFGS_B, SPSA, NELDER_MEAD
from qiskit_algorithms import VQE
#from qiskit.utils import QuantumInstance
from qiskit.primitives import Estimator

optimizer = SLSQP(maxiter=200)

estimator = Estimator()
vqe = VQE(estimator, ansatz, optimizer)
res = vqe.compute_minimum_eigenvalue(qubit_p_op)

import numpy as np


# we will iterate over these different optimizers
optimizers = [COBYLA(maxiter=80), L_BFGS_B(maxiter=60), SLSQP(maxiter=60)]
converge_counts = np.empty([len(optimizers)], dtype=object)
converge_vals = np.empty([len(optimizers)], dtype=object)

for i, optimizer in enumerate(optimizers):
    print("\rOptimizer: {}        ".format(type(optimizer).__name__), end="")
    #algorithm_globals.random_seed = 50
    #ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")

    counts = []
    values = []

    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)

    vqe = VQE(estimator, ansatz, optimizer, callback=store_intermediate_result)
    res = vqe.compute_minimum_eigenvalue(qubit_p_op)
    converge_counts[i] = np.asarray(counts)
    converge_vals[i] = np.asarray(values)

from qiskit_algorithms.eigensolvers import NumPyEigensolver

numpy_solver = NumPyEigensolver()
exact_result = numpy_solver.compute_eigenvalues(qubit_p_op)
ref_value = exact_result.eigenvalues[0]
print(f"Reference value: {ref_value  }")
print(f"VQE values: {res.optimal_value }")

# Chuyển toán tử sang dạng ma trận
from qiskit.quantum_info import SparsePauliOp
Hopt = qubit_p_op # Hamiltonian tĩnh (static)
H_static = Hopt.to_matrix()
print(Hopt)

dipole_ops = dipole.second_q_ops()
print("Nội dung của dipole_ops:", dipole_ops)
# Lấy toán tử moment lưỡng cực từ phương Z (vì X và Y rỗng)
dipole_op = dipole_ops["ZDipole"]
dipole_qubit = mapper.map(dipole_op)
dipole_matrix = dipole_qubit.to_matrix()

# Thiết lập các tham số cho trường ngoài phụ thuộc thời gian
Gamma = 0.25
E0 = 0.01
# Thiết lập các tham số tiến hóa thời gian
T_total = 100    # Tổng thời gian tiến hóa
dt = 0.1         # Bước thời gian nhỏ
num_steps = int(T_total / dt)
num_qubits = Hopt.num_qubits  # số qubit

# Khởi tạo ma trận tiến hóa tổng U_total là ma trận đơn vị
U_total = np.eye(2**num_qubits, dtype=complex)

# Hàm tính Hamiltonian H(t)
def H_t(t, H_static, dipole_matrix, E0, Gamma):
    f_t = (E0 / np.pi) * Gamma / (Gamma**2 + t**2)
    return H_static + f_t * dipole_matrix



Đã thêm moment lưỡng cực vào Hamiltonian.
Coefficients của Hamiltonian: Polynomial Tensor
 "+-":
array([[-1.25633907e+00, -1.37083854e-17],
       [-6.07732712e-17, -4.71896007e-01]])
 "++--":
array([6.75710155e-01, 1.39486891e-16, 1.80931200e-01, 6.64581730e-01,
       2.57172666e-16, 6.98573723e-01])
Fermionic Operator
number spin orbitals=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.33229086512764816 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.33229086512764816 * ( +_0 +_3 -_3 -_0 )
+ 0.0904655998921157 * ( +_0 +_0 -_1 -_1 )
+ 0.0904655998921157 * ( +_0 +_1 -_0 -_1 )
+ 0.0904655998921157 * ( +_0 +_2 -_3 -_1 )
+ 0.0904655998921157 * ( +_0 +_3 -_2 -_1 )
+ 0.0904655998921157 * ( +_1 +_0 -_1 -_0 )
+ 0.0904655998921157 * ( +_1 +_1 -_0 -_0 )
+ 0.0904655998921157 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211

In [13]:
import numpy as np
from scipy.linalg import expm

def calculate_magnus_U(t0, t, H_static, dipole_matrix, E0, Gamma, num_points=20):
    """
    Tính toán toán tử tiến hóa U(t) sử dụng Magnus expansion bậc 2.
    
    Args:
        t0: Thời điểm bắt đầu
        t: Thời điểm kết thúc
        H_static: Ma trận Hamiltonian tĩnh
        dipole_matrix: Ma trận moment lưỡng cực
        E0, Gamma: Các tham số của trường
        num_points: Số điểm lấy mẫu cho tích phân số (mặc định: 20)
        
    Returns:
        U_magnus: Toán tử tiến hóa thời gian theo Magnus expansion bậc 2
    """
    # Hàm tính Hamiltonian H(t)
    def H_t(tau, H_static, dipole_matrix, E0, Gamma):
        f_t = (E0 / np.pi) * Gamma / (Gamma**2 + tau**2)
        return H_static + f_t * dipole_matrix
    
    # Tạo mảng các điểm thời gian
    dt = (t - t0) / num_points
    time_points = np.linspace(t0, t, num_points+1)
    
    # Tính Omega_1 = -i * ∫_t0^t H(τ) dτ
    omega1 = np.zeros_like(H_static, dtype=complex)
    
    for i in range(num_points):
        tau1 = time_points[i]
        tau2 = time_points[i+1]
        
        H_tau1 = H_t(tau1, H_static, dipole_matrix, E0, Gamma)
        H_tau2 = H_t(tau2, H_static, dipole_matrix, E0, Gamma)
        
        # Tích phân theo quy tắc hình thang
        omega1 += 0.5 * (H_tau1 + H_tau2) * dt
    
    omega1 = -1j * omega1
    
    # Tính Omega_2 = -0.5 * ∫_t0^t ∫_t0^τ1 [H(τ1), H(τ2)] dτ2 dτ1
    omega2 = np.zeros_like(H_static, dtype=complex)
    
    for i in range(1, num_points+1):
        tau1 = time_points[i]
        
        for j in range(i):
            tau2 = time_points[j]
            
            # Tính [H(tau1), H(tau2)]
            H_tau1 = H_t(tau1, H_static, dipole_matrix, E0, Gamma)
            H_tau2 = H_t(tau2, H_static, dipole_matrix, E0, Gamma)
            
            commutator = H_tau1 @ H_tau2 - H_tau2 @ H_tau1
            
            # Cộng dồn vào tích phân
            omega2 += commutator * dt * dt
    
    omega2 = -0.5 * omega2
    
    # Tổng hợp Magnus expansion đến bậc 2
    omega = omega1 + omega2
    
    # Tính U(t) = exp(Omega)
    U_magnus = expm(omega)
    
    return U_magnus

def check_unitary(U, tol=1e-10):
    """
    Kiểm tra xem ma trận U có phải là unitary không.
    Ma trận unitary thỏa mãn: U† U = U U† = I
    
    Args:
        U: Ma trận cần kiểm tra
        tol: Dung sai cho phép (do lỗi tính toán số)
        
    Returns:
        is_unitary: True nếu U là ma trận unitary, False nếu không phải
        error: Độ lớn của sai số UU† - I
    """
    # Tính U dagger (liên hợp chuyển vị của U)
    U_dagger = U.conj().T
    
    # Tính UU† và U†U
    UUd = U @ U_dagger
    UdU = U_dagger @ U
    
    # Tạo ma trận đơn vị cùng kích thước
    I = np.eye(U.shape[0], dtype=complex)
    
    # Tính độ lệch so với ma trận đơn vị
    error_UUd = np.linalg.norm(UUd - I, 'fro')
    error_UdU = np.linalg.norm(UdU - I, 'fro')
    max_error = max(error_UUd, error_UdU)
    
    # Kiểm tra tính unitary
    is_unitary = max_error < tol
    
    return is_unitary, max_error

# Kiểm tra với ma trận test
def test_with_sample_matrices():
    # Tạo ma trận test đơn giản
    H_test = np.array([[1.0, 0.5], [0.5, 2.0]], dtype=complex)
    dipole_test = np.array([[0.0, 0.1], [0.1, 0.0]], dtype=complex)
    
    # Các tham số
    t0 = 0.0
    t = 0.1
    E0 = 0.01
    Gamma = 0.25
    
    print("Tính toán U_magnus với ma trận test...")
    U_magnus = calculate_magnus_U(t0, t, H_test, dipole_test, E0, Gamma, num_points=30)
    
    # In ma trận U_magnus
    print("\nU_magnus:")
    print(U_magnus)
    
    # Kiểm tra tính unitary
    is_unitary, error = check_unitary(U_magnus)
    print("\nKiểm tra tính unitary:")
    print(f"U_magnus có phải là ma trận unitary không? {is_unitary}")
    print(f"Độ lớn sai số: {error:.10e}")
    
    # In thêm các ma trận kiểm tra
    print("\nU_magnus U_magnus†:")
    print(U_magnus @ U_magnus.conj().T)
    
    print("\nU_magnus† U_magnus:")
    print(U_magnus.conj().T @ U_magnus)
    
    return U_magnus

# Kiểm tra với ma trận thực tế
def check_actual_matrices():
    try:
        # Giả sử H_static và dipole_matrix đã được định nghĩa
        print("\nKích thước ma trận H_static:", H_static.shape)
        print("Kích thước ma trận dipole_matrix:", dipole_matrix.shape)
        
        # Các tham số
        t0 = 0.0
        t = 0.1
        E0 = 0.01
        Gamma = 0.25
        
        print("\nTính toán U_magnus với ma trận thực tế...")
        U_magnus = calculate_magnus_U(t0, t, H_static, dipole_matrix, E0, Gamma, num_points=30)
        
        # Kiểm tra tính unitary
        is_unitary, error = check_unitary(U_magnus)
        print("\nKiểm tra tính unitary cho ma trận thực tế:")
        print(f"U_magnus có phải là ma trận unitary không? {is_unitary}")
        print(f"Độ lớn sai số: {error:.10e}")
        
        return U_magnus
    except NameError:
        print("\nKhông tìm thấy biến H_static hoặc dipole_matrix. Chỉ thực hiện kiểm tra với ma trận test.")
        return None

# Thực thi
print("Kiểm tra Magnus expansion và tính unitary của toán tử tiến hóa thời gian")
print("=" * 70)

# Trước tiên kiểm tra với ma trận test
test_magnus = test_with_sample_matrices()

# Sau đó thử với ma trận thực tế nếu có
actual_magnus = check_actual_matrices()

Kiểm tra Magnus expansion và tính unitary của toán tử tiến hóa thời gian
Tính toán U_magnus với ma trận test...

U_magnus:
[[ 0.99375986-0.09966652j -0.00748359-0.04951695j]
 [-0.00748391-0.0495169j   0.97882852-0.19846105j]]

Kiểm tra tính unitary:
U_magnus có phải là ma trận unitary không? True
Độ lớn sai số: 3.1428768796e-16

U_magnus U_magnus†:
[[1.00000000e+00+6.24745465e-18j 1.42438637e-18-6.70556078e-18j]
 [1.42438637e-18+5.32371774e-18j 1.00000000e+00-5.75468854e-18j]]

U_magnus† U_magnus:
[[1.00000000e+00-6.21739694e-18j 1.33299409e-18-6.01652418e-18j]
 [1.33299409e-18+7.59031480e-18j 1.00000000e+00+5.75468854e-18j]]

Kích thước ma trận H_static: (16, 16)
Kích thước ma trận dipole_matrix: (16, 16)

Tính toán U_magnus với ma trận thực tế...

Kiểm tra tính unitary cho ma trận thực tế:
U_magnus có phải là ma trận unitary không? True
Độ lớn sai số: 3.6835338847e-16


In [23]:
import numpy as np

# U_total là toán tử tiến hóa bạn đã tính bằng Magnus
identity = np.eye(actual_magnus.shape[0], dtype=complex)
norm = np.linalg.norm(np.dot(actual_magnus.conj().T, actual_magnus) - identity)

print("Norm của (U†U - I):", norm)

Norm của (U†U - I): 3.683533884653161e-16


In [14]:
print("U magnus thuc te", actual_magnus)

U magnus thuc te [[ 9.99998585e-01-0.00168224j  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          9.92430835e-01+0.12279974j
  -9.53349886e-05+0.00111942j  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j         -9.29358142e-05+0.00111962j
   9.99002000e-01+0.04465134j  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j

In [19]:
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
from qiskit.quantum_info import Statevector, Operator

# ... (Phần code của bạn để có H_static và dipole_matrix) ...
# Đảm bảo H_static và dipole_matrix là các ma trận NumPy

print("--- Bắt đầu tính toán U(t) bằng khai triển Magnus bậc 2 ---")

# Thiết lập các tham số (Lấy lại từ code trước nếu cần)
Gamma = 0.25
E0 = 0.01
T_total = 100 # Thời gian tổng
dt = 0.1  # Bước thời gian (Có thể cần nhỏ hơn cho bậc 2 để thấy rõ lợi ích)
num_steps = int(T_total / dt)
num_qubits = Hopt.num_qubits
dim = 2**num_qubits

# Hàm tính Hamiltonian H(t) (Như code trước)
def H_t(t, H_static, dipole_matrix, E0, Gamma):
    if Gamma**2 + t**2 < 1e-15:
       f_t = 0
    else:
       f_t = (E0 / np.pi) * Gamma / (Gamma**2 + t**2)
    if H_static.shape != dipole_matrix.shape:
        raise ValueError(f"Shape mismatch: H_static {H_static.shape}, dipole_matrix {dipole_matrix.shape}")
    return H_static + f_t * dipole_matrix

# 1. Khởi tạo U(t=0) = I
U_t_m2 = np.eye(dim, dtype=complex) # Đặt tên khác để phân biệt
current_time = 0.0

print(f"Kích thước ma trận U: {U_t_m2.shape}")
print(f"Bắt đầu vòng lặp {num_steps} bước...")

# 2. Vòng lặp thời gian
for k in range(num_steps):
    t = current_time
    t_next = t + dt

    # Tính H tại điểm đầu và điểm cuối của khoảng
    try:
        H_start = H_t(t, H_static, dipole_matrix, E0, Gamma)
        H_end = H_t(t_next, H_static, dipole_matrix, E0, Gamma)
    except ValueError as e:
        print(f"Lỗi khi tính H ở bước {k}: {e}")
        break

    # Tính số hạng trung bình (cho Omega1)
    H_avg = (H_start + H_end) / 2.0

    # Tính Commutator [H(t+dt), H(t)]
    try:
        # commutator = H_end @ H_start - H_start @ H_end # [H_end, H_start]
        # Công thức chuẩn là [H(t+dt), H(t)] = H(t+dt)H(t) - H(t)H(t+dt)
        commutator = H_end @ H_start - H_start @ H_end
    except Exception as e:
        print(f"Lỗi khi tính commutator ở bước {k}: {e}")
        break

    # Tính toán tử Magnus xấp xỉ bậc 2 cho bước này
    # Omega_step ≈ -i * dt * H_avg + (dt^2 / 12) * [H_end, H_start]
    # Chú ý dấu: Omega2 = (-i)^2/2 * Integral -> Omega2 approx -1/2 * (dt^2/6) * [H_end, H_start]? Check lại hệ số.
    # Hệ số dt^2/12 cho [H_end, H_start] là phổ biến trong các sơ đồ bậc 2.
    Omega_step = -1j * dt * H_avg + (dt**2 / 12.0) * commutator

    # Tính toán tử tiến hóa cho bước nhỏ bằng scipy.linalg.expm
    try:
        U_step = expm(Omega_step)
    except Exception as e:
        print(f"Lỗi khi tính expm(Omega_step) ở bước {k}: {e}")
        break

    # 3. Cập nhật toán tử tiến hóa tổng
    # U(t + dt) = U_step(t+dt, t) @ U(t, 0)
    try:
        U_t_m2 = U_step @ U_t_m2
    except Exception as e:
        print(f"Lỗi khi nhân ma trận U ở bước {k}: {e}")
        break

    # Cập nhật thời gian
    current_time += dt

    # In tiến trình (tùy chọn)
    if (k + 1) % (num_steps // 10) == 0:
        print(f"Đã hoàn thành {k+1}/{num_steps} bước (Magnus Bậc 2)...")

print(f"Hoàn thành tính toán (Magnus Bậc 2). Thời gian cuối cùng: {current_time:.4f}")

# Ma trận U cuối cùng chính là U(T_total) tính bằng Magnus bậc 2
U_final_m2 = U_t_m2
print(f"Shape của U_final_m2: {U_final_m2.shape}")

# --- Kiểm tra tính Unitarity của U_final_m2 ---
identity_check_m2 = U_final_m2 @ U_final_m2.conj().T
is_unitary_m2 = np.allclose(identity_check_m2, np.eye(dim))
print(f"\nKiểm tra tính Unitarity của U_final_m2:")
print(f"U_final_m2 @ U_final_m2^dagger gần bằng I? {is_unitary_m2}")
if not is_unitary_m2:
    print("Ma trận U (Magnus Bậc 2) không hoàn toàn unitary.")
    max_deviation_m2 = np.max(np.abs(identity_check_m2 - np.eye(dim)))
    print(f"Độ lệch tối đa khỏi ma trận đơn vị: {max_deviation_m2}")

# --- Sử dụng U_final_m2 để tiến hóa trạng thái ban đầu ---
# Lấy psi0 từ VQE như code trước
optimal_params = res.optimal_parameters
initial_circuit = ansatz.assign_parameters(optimal_params)
psi0 = Statevector(initial_circuit).data
psi_final_m2 = U_final_m2 @ psi0

# --- Tính moment lưỡng cực tại thời điểm cuối ---
# Lấy dipole_matrix như code trước
final_dipole_z_m2 = np.real(np.conj(psi_final_m2) @ dipole_matrix @ psi_final_m2)
print(f"\nMoment lưỡng cực Z tại T={T_total:.2f} tính từ U_final_m2: {final_dipole_z_m2}")

# --- So sánh với kết quả Magnus bậc 1 (Midpoint) ---
# Giả sử bạn đã chạy code trước và có U_final từ phương pháp Midpoint
# U_final = ... (Kết quả từ code trước)
# psi_final_magnus = U_final @ psi0
# final_dipole_z_magnus = np.real(np.conj(psi_final_magnus) @ dipole_matrix @ psi_final_magnus)
# print(f"Moment lưỡng cực Z tại T={T_total:.2f} tính từ U_final (Midpoint): {final_dipole_z_magnus}")
# print(f"Độ chênh lệch (M2 vs Midpoint): {np.abs(final_dipole_z_m2 - final_dipole_z_magnus)}")

--- Bắt đầu tính toán U(t) bằng khai triển Magnus bậc 2 ---
Kích thước ma trận U: (16, 16)
Bắt đầu vòng lặp 1000 bước...
Đã hoàn thành 100/1000 bước (Magnus Bậc 2)...
Đã hoàn thành 200/1000 bước (Magnus Bậc 2)...
Đã hoàn thành 300/1000 bước (Magnus Bậc 2)...
Đã hoàn thành 400/1000 bước (Magnus Bậc 2)...
Đã hoàn thành 500/1000 bước (Magnus Bậc 2)...
Đã hoàn thành 600/1000 bước (Magnus Bậc 2)...
Đã hoàn thành 700/1000 bước (Magnus Bậc 2)...
Đã hoàn thành 800/1000 bước (Magnus Bậc 2)...
Đã hoàn thành 900/1000 bước (Magnus Bậc 2)...
Đã hoàn thành 1000/1000 bước (Magnus Bậc 2)...
Hoàn thành tính toán (Magnus Bậc 2). Thời gian cuối cùng: 100.0000
Shape của U_final_m2: (16, 16)

Kiểm tra tính Unitarity của U_final_m2:
U_final_m2 @ U_final_m2^dagger gần bằng I? True

Moment lưỡng cực Z tại T=100.00 tính từ U_final_m2: 2.7755533996235164


In [22]:
import numpy as np

# U_total là toán tử tiến hóa bạn đã tính bằng Magnus
identity = np.eye(U_final_m2.shape[0], dtype=complex)
norm = np.linalg.norm(np.dot(U_final_m2.conj().T, U_final_m2) - identity)

print("Norm của (U†U - I):", norm)

Norm của (U†U - I): 2.2491610255167007e-14


In [20]:
import numpy as np
from scipy.linalg import expm

U_total = np.eye(2**num_qubits, dtype=complex)
times = np.arange(0, T_total, dt)

for idx, t in enumerate(times):
    # Lấy hai điểm thời gian: t và t+dt
    H_t0 = H_t(t, H_static, dipole_matrix, E0, Gamma)
    H_t1 = H_t(t + dt, H_static, dipole_matrix, E0, Gamma)
    # Trung bình tại giữa đoạn
    H_mid = H_t(t + 0.5*dt, H_static, dipole_matrix, E0, Gamma)
    
    # Magnus bậc 2
    Omega1 = -1j * dt * H_mid
    commutator = np.dot(H_t1, H_t0) - np.dot(H_t0, H_t1)
    Omega2 = - (dt**2) / 12 * commutator
    
    Omega = Omega1 + Omega2
    U_step = expm(Omega)
    U_total = U_step @ U_total


In [21]:
import numpy as np

# U_total là toán tử tiến hóa bạn đã tính bằng Magnus
identity = np.eye(U_total.shape[0], dtype=complex)
norm = np.linalg.norm(np.dot(U_total.conj().T, U_total) - identity)

print("Norm của (U†U - I):", norm)


Norm của (U†U - I): 2.0384475563021154e-14


In [24]:
import numpy as np
from scipy.integrate import simps
from scipy.linalg import expm
import matplotlib.pyplot as plt

# Các tham số
Gamma = 0.25
E0 = 0.01
T_total = 100  # fs
dt = 0.1  # fs
n_steps = int(T_total / dt) + 1
times = np.linspace(0, T_total, n_steps)
m_points = 500  # Giảm số điểm tích phân để tối ưu

# Hàm trường ngoài V(t)
def V_t(t, Gamma, E0, dipole_matrix):
    coefficient = (E0 / np.pi) * (Gamma / (Gamma**2 + t**2))
    return coefficient * dipole_matrix

# Hàm H(t)
def H_t(t, H_static, dipole_matrix, Gamma, E0):
    return H_static + V_t(t, Gamma, E0, dipole_matrix)

# Hàm tính commutator
def commutator(A, B):
    return A @ B - B @ A

# Hàm kiểm tra sai số đơn vị tính
def check_unitary_error(U):
    identity = np.eye(U.shape[0])
    error = np.linalg.norm(U.conj().T @ U - identity, ord='fro')
    return error

# Phương pháp Magnus bậc 2 tối ưu
def compute_U_tj_magnus2(H_static, dipole_matrix, t_j, m_points, Gamma, E0):
    if t_j == 0:
        return np.eye(H_static.shape[0], dtype=complex)
    t_points = np.linspace(0, t_j, m_points)
    
    # Tính số hạng bậc 1: -i ∫ H(s) ds
    H_integral = np.zeros_like(H_static, dtype=complex)
    H_t_values = np.array([H_t(t, H_static, dipole_matrix, Gamma, E0) for t in t_points])
    for i in range(H_static.shape[0]):
        for j in range(H_static.shape[1]):
            H_integral[i,j] = simps(H_t_values[:,i,j], t_points)
    Omega1 = -1j * H_integral
    
    # Tính số hạng bậc 2: (-i)^2/2 ∫ ds1 ∫ ds2 [H(s1), H(s2)]
    Omega2 = np.zeros_like(H_static, dtype=complex)
    dt = t_j / (m_points - 1) if m_points > 1 else t_j
    for s1_idx in range(m_points):
        s1 = t_points[s1_idx]
        H_s1 = H_t_values[s1_idx]
        inner_integral = np.zeros_like(H_static, dtype=complex)
        s2_points = t_points[:s1_idx+1]
        for i in range(H_static.shape[0]):
            for j in range(H_static.shape[1]):
                inner_integral[i,j] = simps(H_t_values[:s1_idx+1,i,j], s2_points)
        comm = commutator(H_s1, inner_integral)
        Omega2 += comm * dt
    Omega2 *= (-1j)**2 / 2
    
    # Tổng Omega
    Omega = Omega1 + Omega2
    U = expm(Omega)
    return U

# Tính U(t_j) và kiểm tra độ chính xác
target_unitaries = []
unitary_errors = []

for t_j in times:
    U_tj = compute_U_tj_magnus2(H_static, dipole_matrix, t_j, m_points, Gamma, E0)
    error = check_unitary_error(U_tj)
    target_unitaries.append(U_tj)
    unitary_errors.append(error)

# In sai số
avg_error = np.mean(unitary_errors)
max_error = np.max(unitary_errors)
print("Magnus bậc 2 (tối ưu):")
print(f"  Sai số đơn vị tính trung bình: {avg_error:.2e}")
print(f"  Sai số đơn vị tính tối đa: {max_error:.2e}")

# Vẽ sai số
plt.figure(figsize=(6, 4))
plt.plot(times, unitary_errors, label='Magnus 2')
plt.xlabel("Time (fs)")
plt.ylabel("Unitary Error ($\|U^\dagger U - I\|_F$)")
plt.yscale('log')
plt.title("Unitary Error")
plt.legend()
plt.grid(True)
plt.show()

# Lưu target_unitaries cho GA-VQA
chosen_unitaries = target_unitaries

KeyboardInterrupt: 

In [1]:
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
from qiskit.quantum_info import Statevector
import qiskit
from qoop.compilation.qsp import QuantumStatePreparation

# == Params ==
Gamma = 0.25
E0 = 0.01
T_total = 10
dt = 0.2
num_steps = int(T_total / dt)
time_array = np.linspace(0, T_total, num_steps)
# Số qubit
num_qubits = 2

# == Define H_static and mu ==
from qiskit.quantum_info import Pauli
# 2-qubit system: H_static = ZI + IZ, mu = XI + IX
ZI = Pauli("ZI").to_matrix()
IZ = Pauli("IZ").to_matrix()
XI = Pauli("XI").to_matrix()
IX = Pauli("IX").to_matrix()

H_static = ZI + IZ
mu = XI + IX


# == Magnus Expansion Order 2 ==
def compute_U_magnus_2(H_static, mu, E0, Gamma, t, dt=0.1):
    times = np.arange(0, t, dt)
    Omega1 = np.zeros_like(H_static, dtype=complex)
    Omega2 = np.zeros_like(H_static, dtype=complex)

    # Precompute B_I(s) at each s
    B_I_list = []
    for s in times:
        f_s = (E0 / np.pi) * Gamma / (Gamma**2 + s**2)
        U_As = expm(1j * H_static * s)
        B_I_s = f_s * (U_As @ mu @ U_As.conj().T)
        B_I_list.append(B_I_s)
        Omega1 += B_I_s * dt

    # Compute Omega2
    for i, s1 in enumerate(times):
        for j in range(i):
            s2 = times[j]
            comm = B_I_list[i] @ B_I_list[j] - B_I_list[j] @ B_I_list[i]
            Omega2 += comm * dt * dt

    Omega1 *= -1j
    Omega2 *= -0.5
    U_static = expm(-1j * H_static * t)
    return U_static @ expm(Omega1 + Omega2)


In [ ]:
# Phần code từ biklmjroinha.py (đã được điều chỉnh một chút để dễ tích hợp)
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp # Đảm bảo import SparsePauliOp
import numpy as np

# --- Phần thiết lập H_static và dipole_op_qubit ---
# (Giữ nguyên phần code của bạn để tính H_static và dipole_op_qubit)
# Ví dụ (bạn cần điền đầy đủ phần này từ code của bạn):
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
problem = driver.run()
hamiltonian_static_obj = problem.hamiltonian
second_q_op_static = hamiltonian_static_obj.second_q_op()

mapper = JordanWignerMapper()
H_static_qubit = mapper.map(second_q_op_static) # Đây là H0 dưới dạng SparsePauliOp

dipole_property = problem.properties.electronic_dipole_moment
# Giả sử bạn đã xử lý nuclear_dipole_moment như trong code gốc
# và lấy ra toán tử moment lưỡng cực theo phương Z
dipole_ops_second_q = dipole_property.second_q_ops()
dipole_op_z_second_q = dipole_ops_second_q["ZDipole"] # Hoặc tên key phù hợp
dipole_op_qubit = mapper.map(dipole_op_z_second_q) # Đây là D dưới dạng SparsePauliOp

# Các tham số từ bài báo
Gamma = 0.25
E0 = 0.01
# --- Kết thúc phần thiết lập ---

# Hàm tạo H(t) theo công thức của bài báo
def get_hamiltonian_t_from_paper(t, H_static_qubit_op, dipole_qubit_op, E0, Gamma):
    """
    Tạo Hamiltonian phụ thuộc thời gian H(t) = H_static + f(t) * Dipole.
    Trả về một SparsePauliOp.
    """
    f_t = (E0 / np.pi) * Gamma / (Gamma**2 + t**2)
    # Nhân toán tử Pauli với một hằng số
    time_dependent_part = dipole_qubit_op * f_t
    # Cộng hai toán tử Pauli
    H_total_t = H_static_qubit_op + time_dependent_part
    return H_total_t

# Ví dụ cách sử dụng hàm get_hamiltonian_t_from_paper:
# H_at_time_0 = get_hamiltonian_t_from_paper(0, H_static_qubit, dipole_op_qubit, E0_paper, Gamma_paper)
# print("H(t=0) từ bài báo:\n", H_at_time_0)
# print("Ma trận của H(t=0):\n", H_at_time_0.to_matrix())